<a href="https://colab.research.google.com/github/sebastianvillaf/Quantum-AI-Summit-2025-stats-analysis-for-qiskit-fall-fest-2026/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from sklearn.metrics import matthews_corrcoef
from itertools import groupby
from scipy.stats import chi2_contingency

In [3]:
predata=pd.read_csv("/content/summit.csv")

z=predata.isna()
data=predata.fillna(value="")

In [27]:
day1=(data["Asistencia día 1"]==1).sum()
day2=(data["Asistencia día 2"]==1).sum()
day3=(data["Asistencia día 3"]==1).sum()

print(f"asistieron {day1} personas el Viernes 19")
print(f"asistieron {day2} personas el Sabado 20")
print(f"asistieron {day3} personas el Domingo 21")
data=data.replace({"VERDADERO":1,"FALSO":0}).infer_objects(copy=False)

asistieron 124 personas el Viernes 19
asistieron 55 personas el Sabado 20
asistieron 70 personas el Domingo 21


In [17]:
#Correlación de retención
cr12=matthews_corrcoef(data["Asistencia día 1"],data["Asistencia día 2"])
print(cr12)
#Hubo mucha rotacion entre el dia 1 y 2, posiblemente porque aseguraron 1 de los 2 dias minimos que
#eran necesarios para recibir certificado

cr23=matthews_corrcoef(data["Asistencia día 2"],data["Asistencia día 3"])
print(cr23)

cr13=matthews_corrcoef(data["Asistencia día 1"],data["Asistencia día 3"])
print(cr13)

#entonces esto significa que el nuevo grupo de personas que fue el dia 2 tambien fue al dia 3
#esto podria ser posible porque los del dia 2 necesitaban ir otro dia mas
#para asegurar su certificado (el requisito era ir minimo 2 dias de los 3)
#entonces eso sugiere que definitivamente debemos ofrecer certificado en el qiskit fall fest 2026
#y debemos tener una tecnica de engage para
#que el grupo de personas que asistió al primer dia no dejen de ir al segundo y tercer dia

#Nota: Hay muchas personas que asistieron al primer día y no volvieron a asistir, es posible
#que hayan perdido motivación o la hilación debido a la dificultad de los temas de quantum computing


-0.3488323752494592
0.17023546374960816
-0.259392186072743


In [36]:
data["carrera"] = data["¿Qué carrera estudió, estudia o planea estudiar?"].str.strip().str.lower()
data["carrera"] = data["carrera"].replace({"ciencias de la computacion": "ciencias de la computación","ingenieria de software": "ingeniería de software","ingenieria de sistemas": "ingeniería de sistemas","ingeniera de sistemas":"ingeniería de sistemas","ingeniería física":"física"}, regex=True)

In [39]:
#Engagement según la carrera
data["Total Asistencia"]=data["Asistencia día 1"] + data["Asistencia día 2"] + data["Asistencia día 3"]

ranking_carreras=data.groupby("carrera")["Total Asistencia"].sum().sort_values(ascending=False)

print(ranking_carreras.head(7))

#Esta es la lista de carreras cuyos estudiantes asistieron más a minimo 1 dia del evento

carrera
ciencias de la computación    35
ingeniería de sistemas        32
física                        25
ingeniería de software        24
ingeniería electrónica        10
economía                      10
ingeniería mecatrónica         8
Name: Total Asistencia, dtype: int64


In [41]:
#Prueba chi cuadrado
#H0=Existe una relación (dependencia) entre la carrera y el abandono.
#H1=No existe relación. El abandono afecta a STEM y no STEM por igual
